<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml0606/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/tibaml0606/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/tibaml0606/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f7774302b50>)

In [2]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
predict_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
predict_df

In [ ]:
datas = pd.concat([train_df, predict_df], axis=0, ignore_index=True)
datas = datas.drop(["PassengerId", "Survived"], axis=1)
datas

In [34]:
# 資料預處理
# Step1. 填補空值
# 1.1 check有多少個空值
s = datas.isna().sum()
# s[s!=0]: filter操作
s[s != 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [ ]:
# 1.2 check 每一個欄位是哪一種型態
# a. 數值: Age, SibSp, Parch, Ticket(#share), Fare
# b. 類別: Pclass, Name(mid), Sex, Cabin(first), Embarked